# 自動機械学習 Automated Machine Learning による離反予測モデリング

顧客属性データを用いて離反予測モデルを開発します。
- Python SDK のインポート
- Azure ML service Workspace への接続
- Experiment の作成
- データの準備
- 自動機械学習の事前設定
- モデル学習と結果の確認

## 1. 事前準備
### Python SDK のインポート
Azure Machine Learning service の Python SDKをインポートします

In [1]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

W0919 00:15:42.707293 4527637952 deprecation_wrapper.py:119] From /Users/konabuta/miniconda3/envs/myenv/lib/python3.6/site-packages/azureml/automl/core/_vendor/automl/client/core/common/tf_wrappers.py:36: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0919 00:15:42.708401 4527637952 deprecation_wrapper.py:119] From /Users/konabuta/miniconda3/envs/myenv/lib/python3.6/site-packages/azureml/automl/core/_vendor/automl/client/core/common/tf_wrappers.py:36: The name tf.logging.ERROR is deprecated. Please use tf.compat.v1.logging.ERROR instead.



### Azure ML workspace との接続
Azure Machine Learning service との接続を行います。Azure に対する認証が必要です。

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

azureml	eastus	mlservice	eastus


### 実験名の設定

In [3]:
# choose a name for experiment
experiment_name = 'automl-classification-churn-simple'
experiment=Experiment(ws, experiment_name)

### 学習データの準備

In [4]:
data = "https://amlgitsamples.blob.core.windows.net/churn/CATelcoCustomerChurnTrainingSample.csv"
df = Dataset.Tabular.from_delimited_files(data).to_pandas_dataframe()

In [5]:
df.head()

,age,annualincome,calldroprate,callfailurerate,callingnum,customerid,customersuspended,education,gender,homeowner,...,totalminsusedinlastmonth,unpaidbalance,usesinternetservice,usesvoiceservice,percentagecalloutsidenetwork,totalcallduration,avgcallduration,churn,year,month
0,12,168147,0.06,0.00,4251078442,1,True,Bachelor or equivalent,Male,True,...,15,19,False,False,0.82,5971,663,0,2015,1
1,12,168147,0.06,0.00,4251078442,1,True,Bachelor or equivalent,Male,True,...,15,19,False,False,0.82,3981,995,0,2015,2
2,42,29047,0.05,0.01,4251043419,2,True,Bachelor or equivalent,Female,True,...,212,34,False,True,0.27,7379,737,0,2015,1
3,42,29047,0.05,0.01,4251043419,2,True,Bachelor or equivalent,Female,True,...,212,34,False,True,0.27,1729,432,0,2015,2
4,58,27076,0.07,0.02,4251055773,3,True,Master or equivalent,Female,True,...,216,144,False,False,0.48,3122,624,0,2015,1


In [6]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["churn"],axis=1)
y = df["churn"].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=100,stratify=y)

## 2. 自動機械学習 Automated Machine Learning
### 学習事前設定

In [7]:
automl_settings = {
    "iteration_timeout_minutes": 5,
    "iterations": 5,
    "n_cross_validations": 2,
    "primary_metric": 'AUC_weighted',
    "preprocess": True,
    "enable_voting_ensemble": False,
    "enable_stack_ensemble": False
}

automl_config = AutoMLConfig(task = 'classification',
                             X = X_train,
                             y = y_train,
                             **automl_settings
                            )

### 実行と結果確認

In [8]:
local_run = experiment.submit(automl_config, show_output = True)

Running on local machine
Parent Run ID: AutoML_fdbec0cf-5b3e-4d57-b6c3-496a642470e8
Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS SUMMARY:
For more details, use API: run.get_guardrails()

TYPE:         Missing Values Imputation
STATUS:       PASSED
DESCRIPTION:  There were no missing values found in the training data.

TYPE:         High Cardinality Feature Detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.

**************************************************************

In [9]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [11]:
best_run, fitted_model = local_run.get_output()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-classification-churn-simple,AutoML_fdbec0cf-5b3e-4d57-b6c3-496a642470e8_2,,Completed,Link to Azure Portal,Link to Documentation
